In [1]:
import json
import math

from os.path import join as _join
from subprocess import Popen, PIPE

from wepppy.all_your_base.geo import GeoTransformer, wgs84_proj4

import ee
import numpy as np
import geetools

geodata_dir = '/geodata/'


def safe_float_parse(x):
    """
    Tries to parse {x} as a float. Returns None if it fails.
    """
    try:
        return float(x)
    except:
        return None


def safe_int_parse(x):
    """
    Tries to parse {x} as a int. Returns None if it fails.
    """
    try:
        return int(x)
    except:
        return None


In [2]:
ee.Initialize()

In [3]:
!wget https://palouseresidue.nkn.uidaho.edu/boundaries/whitman_huc12_wgs84.geojson

--2022-04-26 12:15:45--  https://palouseresidue.nkn.uidaho.edu/boundaries/whitman_huc12_wgs84.geojson
Resolving palouseresidue.nkn.uidaho.edu (palouseresidue.nkn.uidaho.edu)... 129.101.202.243
Connecting to palouseresidue.nkn.uidaho.edu (palouseresidue.nkn.uidaho.edu)|129.101.202.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3675606 (3.5M)
Saving to: ‘whitman_huc12_wgs84.geojson.3’

whitman_huc12_wgs84 100%[===================>]   3.50M  --.-KB/s    in 0.02s   

2022-04-26 12:15:45 (211 MB/s) - ‘whitman_huc12_wgs84.geojson.3’ saved [3675606/3675606]



In [4]:
!ogrinfo whitman_huc12_wgs84.geojson

INFO: Open of `whitman_huc12_wgs84.geojson'
      using driver `GeoJSON' successful.
1: whitman_huc12_wgs84_2 (Multi Polygon)


In [5]:
def geojson_bbox(geojson_fn):
    with open(geojson_fn) as fp:
        geojson = json.load(fp)

    coords = []
    for feature in geojson['features']:
        coordinates = feature['geometry']['coordinates']
        try:
            coords.append(np.array(coordinates)[0,0,:,:])
        except:
            pass

    coords = np.concatenate(coords)
    coords.shape

    bbox = np.min(coords[:, 0]), np.max(coords[:, 1]), \
           np.max(coords[:, 0]), np.min(coords[:, 1])
        
    return bbox
            
bbox = geojson_bbox('whitman_huc12_wgs84.geojson')
bbox

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


(-118.249404, 47.260029, -117.039413, 46.417051)

In [6]:
ee_bbox = ee.Geometry.BBox(west=bbox[0], north=bbox[1], east=bbox[2], south=bbox[3])

In [7]:
ee_bbox.toGeoJSON()

{'coordinates': [[[-118.249404, 47.260029],
   [-118.249404, 46.417051],
   [-117.039413, 46.417051],
   [-117.039413, 47.260029]]],
 'geodesic': False,
 'type': 'Polygon'}

In [8]:
# ## Define an ImageCollection
site = ee.Geometry.Point([-72, -42]).buffer(1000)
collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(site).limit(3)

# Set parameters
bands = ['B2', 'B3', 'B4']
scale = 30
name_pattern = '{sat}_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d}'
## the keywords between curly brackets can be {system_date} for the date of the
## image (formatted using `date_pattern` arg), {id} for the id of the image
## and/or any image property. You can also pass extra keywords using the `extra`
## argument. Also, numeric values can be formatted using a format string (as
## shown in {WRS_PATH:%d} (%d means it will be converted to integer)
date_pattern = 'ddMMMy' # dd: day, MMM: month (JAN), y: year
folder = 'MYFOLDER'
data_type = 'uint32'
extra = dict(sat='L8SR')
region = site

# ## Export
tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=collection,
            folder=folder,
            region=site,
            namePattern=name_pattern,
            scale=scale,
            dataType=data_type,
            datePattern=date_pattern,
            extra=extra,
            verbose=True,
            maxPixels=int(1e13)
        )

exporting L8SR_07Apr2013_232-89 to folder 'MYFOLDER' in GDrive
exporting L8SR_21Apr2013_232-89 to folder 'MYFOLDER' in GDrive
exporting L8SR_23May2013_232-89 to folder 'MYFOLDER' in GDrive


In [9]:
year = 2016

dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
            .filterDate(f'{year}-01-01', f'{year}-12-31') \
            .filterBounds(geometry=ee_bbox)
            


In [10]:
median = dataset.median()
band_names = median.bandNames()
bands = band_names.getInfo()
bands

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [11]:
scale = 90
name_pattern = '{sat}_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d}'
date_pattern = 'ddMMMy' # dd: day, MMM: month (JAN), y: year
folder = 'MYFOLDER'
data_type = 'uint32'
extra = dict(sat='L8SR')
region = ee_bbox

In [12]:
from geetools import tools
from geetools.batch import utils as tool_utils

def toLocal(image, name=None, path=None, scale=None, region=None,
            dimensions=None, toFolder=True, bands=None):
    """ Download an Image to your hard drive
    :param image: the image to download
    :type image: ee.Image
    :param path: the path to download the image. If None, it will be
        downloaded to the same folder as the script is
    :type path: str
    :param name: name of the file
    :type name: str
    :param scale: scale of the image to download. If None, tries to get it.
    :type scale: int
    :param region: region to from where to download the image. If None,
        will be the image region
    :type region: ee.Geometry
    :param
    """
    # TODO: checkExist
    try:
        import zipfile
    except:
        raise ValueError(
            'zipfile module not found, install it using '
            '`pip install zipfile`')

    name = name if name else image.id().getInfo()

    scale = scale if scale else int(tools.image.minscale(image).getInfo())

    if region:
        region = tools.geometry.getRegion(region)
    else:
        region = tools.geometry.getRegion(image)

    params = {'region': region,
              'scale': scale}

    if dimensions:
        params.update({'dimensions': dimensions})
        
    if bands:
        params.update({'bands': bands})

    print(params)
    url = image.getDownloadURL(params)

    ext = 'zip'

    tool_utils.downloadFile(url, name, ext)

    filename = '{}.{}'.format(name, ext)

    original_filepath = os.path.join(os.getcwd(), filename)

    if path:
        filepath = os.path.join(path, filename)
        os.rename(original_filepath, filepath)
    else:
        path = os.getcwd()
        filepath = os.path.join(path, filename)

    try:
        zip_ref = zipfile.ZipFile(filepath, 'r')

        if toFolder:
            finalpath = os.path.join(path, name)
        else:
            finalpath = path

        zip_ref.extractall(finalpath)
        zip_ref.close()
    except:
        raise


In [13]:
import os
path = os.getcwd()
path

'/workdir/wepppy/wepppy/gee'

In [15]:
toLocal(
            image=median,
            name=f'{year}_sr8_median',
            scale=scale,
            region=region,
            bands=[bands[0]]
        )

{'region': [[[-118.249404, 46.417051], [-117.039413, 46.417051], [-117.039413, 47.260029], [-118.249404, 47.260029], [-118.249404, 46.417051]]], 'scale': 90, 'bands': ['SR_B1']}
path /workdir/wepppy/wepppy/gee


In [ ]:
!ls

In [64]:
tasks = geetools.batch.image.toLocal(
            image=median,
            scale=scale,
            region=region
        )



EEException: Total request size (2402949969 bytes) must be less than or equal to 50331648 bytes.

In [16]:

import geebap
from geetools import tools

import pprint
pp = pprint.PrettyPrinter(indent=2)

# SEASON
a_season = geebap.season.Season('11-15', '03-15')

# MASKS
cld_mask = geebap.masks.Mask()

# Combine masks in a tuple
masks = (cld_mask,)

# FILTERS
filt_cld = geebap.filters.CloudCover()
# filt_mask = geebap.filters.MaskCover() # Doesn't work

# Combine filters in a tuple
filters = (filt_cld,)#, filt_mask)

# SCORES
best_doy = geebap.scores.Doy('01-15', a_season)
sat = geebap.scores.Satellite()
out = geebap.scores.Outliers(("ndvi",))
ind = geebap.scores.Index("ndvi")
maskpercent = geebap.scores.MaskPercentKernel()
dist = geebap.scores.CloudDist()

# Combine scores in a tuple
scores = (
    best_doy,
    sat,
    out,
    ind,
    maskpercent,
    dist
)

# BAP OBJECT
BAP = geebap.Bap(range=(0, 0),
                 season=a_season,
                 masks=masks,
                 scores=scores,
                 filters=filters)

# SITE
site = ee.Geometry.Polygon([[-71.5,-42.5],
                            [-71.5,-43],
                            [-72,-43],
                            [-72,-42.5]])

# COMPOSITE
composite = BAP.build_composite_best(2019, site=site, indices=("ndvi",))

# `composite` is a ee.Image object, so you can do anything
# from here..
one_value = tools.image.getValue(composite,
                                 site.centroid(),
                                 30, 'client')
pp.pprint(one_value)

ModuleNotFoundError: No module named 'geetools.collection'